In [1]:
import pandas as pd
import logging
import os
import sys
from dotenv import load_dotenv
import pyodbc 

sys.path.insert(1, r'C:\Users\int.matheus\Desktop\DW\Data-Warehouse\Func')
from utils2 import clear_env_variables, load_env, connect_to_database, execute_query_sql

In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def process_query(env_file_path, query_file_path):
    clear_env_variables()
    env_vars = load_env(env_file_path)
    required_vars = ['SERVER', 'DATABASE', 'USERNAME', 'PASSWORD', 'DRIVER', 'PORT']
    
    if all([env_vars.get(var) for var in required_vars]):
        try:
            conn = connect_to_database(env_vars)
            
            if conn:
                try:
                    df = execute_query_sql(conn, query_file_path)
                    conn.close()
                    logging.info(f"DataFrame criado com {len(df)} linhas.")
                    return df
                
                except Exception as e:
                    logging.error(f"Erro ao executar a consulta SQL: {str(e)}")
                    return pd.DataFrame()
            
            else:
                logging.error("Falha na conexão ao banco de dados.")
                return pd.DataFrame()
        
        except pyodbc.Error as ex:
            logging.error(f"Erro ao conectar ao banco de dados: {str(ex)}")
            return pd.DataFrame()
    
    else:
        logging.error("Erro: Não foi possível carregar todas as variáveis de ambiente necessárias.")
        return pd.DataFrame()

In [3]:
# TAKS_01
env_file_path_Task_01 = r'C:\Users\int.matheus\Desktop\DW\Data-Warehouse\Environment\.Env_07'
query_file_path_Task_01 = r'C:\Users\int.matheus\Desktop\DW\Data-Warehouse\Source\ACTYON\ACIONA_CRM\ACIONA_CRM.SQL'

# Processar a TAKS_01
df_task_01 = process_query(env_file_path_Task_01, query_file_path_Task_01)

# TAKS_02
env_file_path_Task_02 = r'C:\Users\int.matheus\Desktop\DW\Data-Warehouse\Environment\.Env_08'
query_file_path_Task_02 = r'C:\Users\int.matheus\Desktop\DW\Data-Warehouse\Source\ACTYON\ACIONA_CRM\ACIONA_CRM.SQL'

# Processar a TAKS_02
df_task_02 = process_query(env_file_path_Task_02, query_file_path_Task_02)

# TAKS_03
env_file_path_Task_03 = r'C:\Users\int.matheus\Desktop\DW\Data-Warehouse\Environment\.Env_09'
query_file_path_Task_03 = r'C:\Users\int.matheus\Desktop\DW\Data-Warehouse\Source\ACTYON\ACIONA_CRM\ACIONA_CRM.SQL'

# Processar a TAKS_03
df_task_03 = process_query(env_file_path_Task_03, query_file_path_Task_03)

# TAKS_04
env_file_path_Task_04 = r'C:\Users\int.matheus\Desktop\DW\Data-Warehouse\Environment\.Env_10'
query_file_path_Task_04 = r'C:\Users\int.matheus\Desktop\DW\Data-Warehouse\Source\ACTYON\ACIONA_CRM\ACIONA_CRM.SQL'

# Processar a TAKS_04
df_task_04 = process_query(env_file_path_Task_04, query_file_path_Task_04)


# Combinar os DataFrames
combined_df = pd.concat([df_task_01, df_task_02,df_task_03,df_task_04], ignore_index=True)

# Exibir o DataFrame combinado
logging.info(f"DataFrame combinado criado com {len(combined_df)} linhas.")
display(combined_df)


2024-07-02 14:40:33,719 - INFO - Variáveis de ambiente carregadas: {'SERVER': '45.77.94.104', 'DATABASE': 'dbActyon_Tim_RPL', 'USERNAME': 'mis', 'PASSWORD': 'mis@123', 'DRIVER': '{ODBC Driver 17 for SQL Server}', 'PORT': '1433'}
2024-07-02 14:40:48,860 - ERROR - Erro ao conectar ao banco de dados: ('08001', '[08001] [Microsoft][ODBC Driver 17 for SQL Server]Provedor TCP: O tempo limite de espera foi atingido.\r\n (258) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 17 for SQL Server]O tempo limite do logon expirou (0); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Erro relatado pela rede ou específico à instância ao estabelecer conexão com o SQL Server. O servidor não foi encontrado ou não está acessível. Verifique se o nome da instância está correto e se o SQL Server está configurado para permitir conexões remotas. Para obter mais informações, consulte os Manuais Online do SQL Server. (258)')
2024-07-02 14:40:48,862 - ERROR - Falha na conexão ao banco de dados.
2024-07-02 14:

""


In [ ]:
with DAG('DAG_EXACT_SDR_LEADS.py', start_date=datetime(2023,10,19), 
         schedule='07 * * * *', catchup=False) as dag:

    t_captura_conta_dados = PythonOperator(
        task_id='captura_conta_dados',
        python_callable=captura_conta_dados
    )

    t_e_valida = BranchPythonOperator(
        task_id='e_valida',
        python_callable=e_valida
    )

    t_valido = BashOperator(
        task_id='valido',
        bash_command="echo 'Quantidade Ok'"
    )

    t_nvalido = BashOperator(
        task_id='nvalido',
        bash_command="echo 'Quantidade Não Ok'"
    )


t_captura_conta_dados >> t_e_valida >> [t_valido, t_nvalido]